In [1]:
import pickle
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from loguru import logger as log
import configparser
import uuid
import os

In [7]:
config = configparser.ConfigParser()
config.read('../config.ini')

['../config.ini']

In [8]:
qdrant_client = QdrantClient(url=config[os.getenv('ENV')]['QdrantUrl'])

if not qdrant_client.collection_exists(config[os.getenv('ENV')]['CollectionName']):
        
        qdrant_client.create_collection(
        collection_name=config[os.getenv('ENV')]['CollectionName'],
        vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    )

In [9]:
with open("../../data/data.pkl", "rb") as f:
    products = pickle.load(f)

In [10]:
for product in products:
    points = [
            PointStruct(
                id=uuid.uuid4().__str__(), 
                vector=embedding.tolist(),
                payload={**product[0]},
            )
            for embedding in product[1]
        ]
    
    qdrant_client.upsert(collection_name=config[os.getenv('ENV')]['CollectionName'], points=points)
    log.info('embedding of product {} added to qdrant successfully',str(product[0]['id']))

2024-11-29 16:18:46.236 | INFO     | __main__:<module>:12 - embedding of product 2089163 added to qdrant successfully
2024-11-29 16:18:46.327 | INFO     | __main__:<module>:12 - embedding of product 2089164 added to qdrant successfully
2024-11-29 16:18:46.423 | INFO     | __main__:<module>:12 - embedding of product 2058607 added to qdrant successfully
2024-11-29 16:18:46.467 | INFO     | __main__:<module>:12 - embedding of product 2089165 added to qdrant successfully
2024-11-29 16:18:46.507 | INFO     | __main__:<module>:12 - embedding of product 2089166 added to qdrant successfully
2024-11-29 16:18:46.537 | INFO     | __main__:<module>:12 - embedding of product 2089167 added to qdrant successfully
2024-11-29 16:18:46.567 | INFO     | __main__:<module>:12 - embedding of product 2089168 added to qdrant successfully
2024-11-29 16:18:46.597 | INFO     | __main__:<module>:12 - embedding of product 2089169 added to qdrant successfully
2024-11-29 16:18:46.619 | INFO     | __main__:<module>:1